# IndoBERT QAT (Quantization-Aware Training) Pipeline

This notebook fine-tunes IndoBERT with Quantization-Aware Training (QAT) for INT8 quantization.

**QAT vs PTQ:**
- QAT: Quantization simulation during training (better accuracy)
- PTQ: Quantization applied after training (faster, simpler)

**Combined Dataset Sources:**
1. Alvin Hanafie - Binary sentiment
2. TikTok Shop - Tokopedia reviews
3. Instagram Cyberbullying - Comment sentiment
4. Tweet TV - TV show sentiment
5. Tweet Film - Movie opinion sentiment

**Pipeline:**
1. Load FP32 model
2. Prepare model for QAT (insert fake quantization ops)
3. Fine-tune with fake quantization
4. Convert to INT8
5. Evaluate quantized model

**Hyperparameters:**
- Learning rate: 2e-5
- Batch size: 16
- Epochs: 3
- Quantization: INT8 (simulated during training)
- Optimizer: AdamW
- Max sequence length: 128

In [ ]:
# !pip install --upgrade numpy
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install transformers datasets accelerate
# !pip install scikit-learn

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.quantization as quantization
import os

## Load Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
print(f"Tokenizer loaded: {tokenizer.__class__.__name__}")

## Define Tokenization Function

In [ ]:
def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

## Load Combined Dataset

In [ ]:
dataset = load_dataset(
    'csv',
    data_files={
        'train': '../datasets/combined/train.csv',
        'validation': '../datasets/combined/val.csv',
        'test': '../datasets/combined/test.csv'
    }
)

print(f"Train samples: {len(dataset['train']):,}")
print(f"Validation samples: {len(dataset['validation']):,}")
print(f"Test samples: {len(dataset['test']):,}")
print(f"\nColumns: {dataset['train'].column_names}")
print(f"\nSample: {dataset['train'][0]}")

## Tokenize Dataset

In [ ]:
tokenized_dataset = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=['text', 'source']
)

print("Tokenization complete!")
print(f"Columns after tokenization: {tokenized_dataset['train'].column_names}")
print(f"\nSample tokenized data:")
print(f"  input_ids length: {len(tokenized_dataset['train'][0]['input_ids'])}")
print(f"  attention_mask length: {len(tokenized_dataset['train'][0]['attention_mask'])}")

## Load FP32 Model and Prepare for QAT

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p1",
    num_labels=2
)

print(f"FP32 Model loaded: {model.__class__.__name__}")
print(f"Number of parameters: {model.num_parameters():,}")

# Configure for QAT
model.qconfig = quantization.get_default_qat_qconfig('fbgemm') # INT8

# Prepare model for QAT (inserts fake quantization modules)
model_qat = quantization.prepare_qat(model, inplace=False)

print(f"\nModel prepared for QAT")
print(f"Fake quantization modules inserted")
print(f"Model will simulate INT8 quantization during training")

## Define Metrics Function

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

## Configure QAT Training Arguments

In [ ]:
output_dir = "./results/indobert-qat-combined"

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir=f"{output_dir}/logs",
    logging_steps=100,
    report_to="none",
    fp16=False,
    push_to_hub=False
)

print("QAT Training arguments configured:")
print(f"  Output directory: {output_dir}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Quantization: INT8 (fake quant during training)")

## Initialize Trainer for QAT

In [ ]:
trainer = Trainer(
    model=model_qat,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

print("QAT Trainer initialized successfully!")

## Start QAT Training

In [ ]:
print("Starting QAT training...")
print("Note: Training with fake quantization may be 10-20% slower than FP32")
print("="*70)

train_result = trainer.train()

print("\n" + "="*70)
print("QAT Training completed!")
print(f"Training loss: {train_result.training_loss:.4f}")
print(f"Training runtime: {train_result.metrics['train_runtime']:.2f} seconds")
print(f"Training samples/second: {train_result.metrics['train_samples_per_second']:.2f}")

## Convert to INT8

After QAT training, convert the model from fake quantization to actual INT8 quantization.

In [ ]:
model_qat.eval()
model_int8 = quantization.convert(model_qat, inplace=False)

print("Model converted to INT8")
print(f"INT8 model parameters: {model_int8.num_parameters():,}")
print("\nModel now uses actual INT8 operations instead of fake quantization")

## Evaluate INT8 Model on Test Set

In [ ]:
trainer_int8 = Trainer(
    model=model_int8,
    args=training_args,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

print("Evaluating QAT INT8 model on test set...")
test_results = trainer_int8.evaluate(tokenized_dataset['test'])

print("\nQAT INT8 Test Set Results:")
print("="*70)
print(f"  Accuracy:  {test_results['eval_accuracy']:.4f}")
print(f"  Precision: {test_results['eval_precision']:.4f}")
print(f"  Recall:    {test_results['eval_recall']:.4f}")
print(f"  F1 Score:  {test_results['eval_f1']:.4f}")
print("="*70)

In [ ]:
predictions_output = trainer.predict(tokenized_dataset['test'])
y_pred = predictions_output.predictions.argmax(-1)
y_true = predictions_output.label_ids
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_true), yticklabels=np.unique(y_true))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

## Save QAT INT8 Model

In [ ]:
save_path_qat = "./models/indobert-qat-int8-combined"
os.makedirs(save_path_qat, exist_ok=True)

torch.save(model_int8.state_dict(), f"{save_path_qat}/model_int8.pth")
tokenizer.save_pretrained(save_path_qat)

print(f"QAT INT8 model saved to: {save_path_qat}")
print("\nTo load the model later:")
print(f"  model = AutoModelForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=2)")
print(f"  model.load_state_dict(torch.load('{save_path_qat}/model_int8.pth'))")
print(f"  tokenizer = AutoTokenizer.from_pretrained('{save_path_qat}')")

## Model Size Comparison

In [ ]:
import os

int8_size = os.path.getsize(f"{save_path_qat}/model_int8.pth") / (1024 * 1024)

print("Model Size Comparison:")
print("="*70)
print(f"QAT INT8 model: {int8_size:.2f} MB")
print(f"Expected FP32 size: ~{int8_size * 4:.2f} MB")
print(f"Size reduction: ~{(1 - int8_size / (int8_size * 4)) * 100:.1f}%")
print("="*70)